# imports + helpers

In [2]:
import sys
import os
PROJECT_ROOT = os.path.abspath("..")
sys.path.append(PROJECT_ROOT)

import json
import sqlite3
from pathlib import Path

import numpy as np
import pandas as pd
import joblib

from sklearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid


from machine_learning.data_collectors import (
    build_ml_dataframe,
    build_supervised_dataset,
    time_split_masks,
    purged_ts_cv_splits,
    TARGET_HORIZONS,
    TARGET_LOOKBACKS,
)
from machine_learning.artifacts import load_mlp_artifact, predict_artifact_to_compare
from machine_learning.evaluators import calculate_deadzone
from machine_learning.artifacts import discover_artifacts_by_horizon
from machine_learning.evaluators import eval_regression, eval_regression_extended
from database_tier1 import TARGET_STOCKS

# XGBoost
from xgboost import XGBRegressor
import xgboost as xgb

# Tus funciones existentes (ya están en tu proyecto)
# build_ml_dataframe
# build_supervised_dataset
# eval_regression_extended
# predict_artifact_to_compare


# ---------- helpers ----------
def _json_default(obj):
    """Para poder json.dump con numpy/pandas timestamps."""
    if isinstance(obj, (np.integer,)):
        return int(obj)
    if isinstance(obj, (np.floating,)):
        return float(obj)
    if isinstance(obj, (np.ndarray,)):
        return obj.tolist()
    if isinstance(obj, (pd.Timestamp,)):
        return obj.isoformat()
    if hasattr(obj, "item"):
        try:
            return obj.item()
        except Exception:
            pass
    return str(obj)

def _safe_write_json(path: Path, payload: dict):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2, ensure_ascii=False, default=_json_default)

def _save_df(path: Path, df: pd.DataFrame):
    path.parent.mkdir(parents=True, exist_ok=True)
    # Prefer parquet si está disponible; sino CSV
    try:
        df.to_parquet(path, index=False)
    except Exception:
        df.to_csv(path.with_suffix(".csv"), index=False)

def save_sklearn_pipeline_artifact(
    run_dir: str,
    pipeline,
    *,
    config: dict,
    metrics: dict,
    feature_names: list[str],
    pred_df_val: pd.DataFrame | None = None,
    pred_df_test: pd.DataFrame | None = None,
    extra: dict | None = None,
):
    """
    Guarda un artefacto estilo 'sklearn' compatible con tu auto-loader:
      - pipeline.joblib
      - feature_names.json
      - config.json
      - metrics.json
      - pred_df_val.parquet (opcional)
      - pred_df_test.parquet (opcional)
      - extra.json (opcional)
    """
    p = Path(run_dir)
    p.mkdir(parents=True, exist_ok=True)

    joblib.dump(pipeline, p / "pipeline.joblib")

    _safe_write_json(p / "feature_names.json", list(feature_names))
    _safe_write_json(p / "config.json", config)
    _safe_write_json(p / "metrics.json", metrics)

    if pred_df_val is not None:
        _save_df(p / "pred_df_val.parquet", pred_df_val)

    if pred_df_test is not None:
        _save_df(p / "pred_df_test.parquet", pred_df_test)

    if extra is not None:
        _safe_write_json(p / "extra.json", extra)

    return str(p)


[DB_PATH_DEFAULT] /home/aprohack/Desktop/all_folders/Investings_project/app/data/stock_data.db
{'foreign_keys': 1, 'journal_mode': 'wal', 'synchronous': 1}


# Parameters

In [7]:
# -----------------------
# Config general del experimento
# -----------------------
from python_scripts.LLM_analysis.preprocess_store_database import get_connection

conn = get_connection()

timeframe = "1Day"
symbols = TARGET_STOCKS

start = None
end = None

include_indicators = True
indicator_names = []
indicator_names = ['RSI_14', 'BBB_20_2.0', 'BBP_20_2.0', 'ATRr_14']

include_economic_indicators = False
econ_indicator_names = []
# econ_indicator_names = ['CPI', 'UNEMPLOYMENT']

include_fmp = False
fmp_feature_names = []
keep_fmp_asof_date = False

# -----------------------
# ELIGE LOOKBACK AQUÍ
# -----------------------
lookback = TARGET_LOOKBACKS[-1]  # <-- cámbialo

# 3 horizontes baseline (puedes editar)
horizons = [60]

# Lags control (opcional):
# - Si lags_by_feature=None => TODOS los base_feature_cols usan 0..lookback-1 (puede explotar columnas si lookback grande).
# - Si quieres algo más manejable para XGB, ejemplo:
#     - OHLCV con lookback completo
#     - indicadores/econ/fmp solo lag0
lags_by_feature = {
    "close": lookback,     
    "volume": lookback,
}
default_lags = 0

# Ejemplo recomendado si tus indicadores/econ/fmp son muchos:
# lags_by_feature = {"open": lookback, "high": lookback, "low": lookback, "close": lookback, "volume": lookback}
# default_lags = 1  # resto solo lag0

print(f"lb={lookback}, h={horizons}")


lb=252, h=[60]


# Dataset construction

In [8]:

# -----------------------
# Construir el dataframe ML
# -----------------------
df = build_ml_dataframe(
    conn=conn,
    symbols=symbols,
    timeframe=timeframe,
    start=start,
    end=end,
    include_indicators=include_indicators,
    indicator_names=indicator_names,
    include_econ=include_economic_indicators,
    econ_indicator_names=econ_indicator_names,
    include_fmp=include_fmp,
    fmp_feature_names=fmp_feature_names,
    fmp_prefix="fmp",                 # consistente con tu MLP config
    keep_fmp_asof_date=keep_fmp_asof_date,
)

df = df.sort_values(["symbol", "timestamp"]).reset_index(drop=True)

# Base features: todo lo numérico excepto columnas "id"
non_feature_cols = {"symbol", "timestamp", "timeframe"}
base_feature_cols = [c for c in df.columns if c not in non_feature_cols]

# Mantén solo numéricas (evita columnas tipo fecha/string si existieran)
base_feature_cols = [c for c in base_feature_cols if pd.api.types.is_numeric_dtype(df[c])]

# (Opcional) normaliza bools a int
for c in base_feature_cols:
    if df[c].dtype == bool:
        df[c] = df[c].astype(np.int8)

print("df shape:", df.shape)
print("n base_feature_cols:", len(base_feature_cols))
print("example cols:", base_feature_cols[:12])

df shape: (204334, 14)
n base_feature_cols: 11
example cols: ['open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap', 'RSI_14', 'BBB_20_2.0', 'BBP_20_2.0', 'ATRr_14']


In [9]:
def xgb_predict_best_iteration(model: XGBRegressor, X: pd.DataFrame) -> np.ndarray:
    """
    Predice respetando best_iteration si hubo early stopping.
    Soporta versiones nuevas y viejas de xgboost.
    """
    # xgboost >= 1.6: iteration_range
    try:
        bi = getattr(model, "best_iteration", None)
        if bi is not None:
            return model.predict(X, iteration_range=(0, int(bi) + 1))
    except TypeError:
        pass

    # xgboost <= 1.5: ntree_limit con best_ntree_limit
    bntl = getattr(model, "best_ntree_limit", None)
    if bntl is not None:
        return model.predict(X, ntree_limit=int(bntl))

    return model.predict(X)

def pick_score(metrics_val: dict) -> float:
    """
    Score para seleccionar hiperparámetros:
    prioridad RankIC cross-sectional diario, fallback spread sharpe, etc.
    """
    # Ajusta estas keys si tu eval_regression_extended usa nombres distintos
    for k in ["DailyRankIC_mean", "QuantileSpread_sharpe", "SpearmanCorr(RankIC)", "PearsonCorr(IC)"]:
        v = metrics_val.get(k, None)
        if v is not None and np.isfinite(v):
            return float(v)
    return float("nan")


In [10]:
# -----------------------
# Hiperparámetros: grid barato y razonable
# -----------------------
xgb_base = dict(
    objective="reg:squarederror",
    n_estimators=5000,
    # learning_rate=0.05,
    # max_depth=6,
    # subsample=0.8,
    # colsample_bytree=0.8,
    n_jobs=-1,
    random_state=42,
    tree_method="hist",

    # <-- aquí (no en fit)
    eval_metric="rmse",
    early_stopping_rounds=200,
)

param_grid = list(ParameterGrid({
    "max_depth": [4, 6],
    "learning_rate": [0.03, 0.07],
    "min_child_weight": [1, 5],
    "subsample": [0.8],
    "colsample_bytree": [0.8],
    "reg_lambda": [1.0],
    # "gamma": [0.0, 1.0],       # (opcional) más regularización
}))

# -----------------------
# Entrenamiento por horizonte
# -----------------------
results = []
saved_run_dirs = []

for horizon in horizons:
    print("\n" + "="*90)
    print(f"HORIZON = {horizon} | lookback = {lookback}")

    X, y, meta = build_supervised_dataset(
        df=df,
        feature_cols=base_feature_cols,
        lookback=lookback,
        horizon=horizon,
        price_col="close",
        group_col="symbol",
        timestamp_col="timestamp",
        lags_by_feature=lags_by_feature,
        default_lags=default_lags,
    )

    # Meta datetime
    meta = meta.copy()
    meta["timestamp"] = pd.to_datetime(meta["timestamp"])
    meta["target_timestamp"] = pd.to_datetime(meta["target_timestamp"])

    # Limpieza básica
    X = X.replace([np.inf, -np.inf], np.nan)

    # Split por target_timestamp (sin leakage por horizon)
    tr_mask, va_mask, te_mask, train_end, val_end= time_split_masks(meta, train_frac=0.65, val_frac=0.17)

    X_train, y_train, meta_train = X.loc[tr_mask], y.loc[tr_mask], meta.loc[tr_mask]
    X_val, y_val, meta_val = X.loc[va_mask], y.loc[va_mask], meta.loc[va_mask]
    X_test, y_test, meta_test = X.loc[te_mask], y.loc[te_mask], meta.loc[te_mask]

    

    # (Opcional) float32 para memoria/velocidad
    X_train = X_train.astype(np.float32)
    X_val = X_val.astype(np.float32)
    X_test = X_test.astype(np.float32)

    feature_names = list(X_train.columns)

    print("Shapes:",
          "train", X_train.shape,
          "val", X_val.shape,
          "test", X_test.shape,
          "| cut:", train_end.date(), val_end.date())

    best = None  # dict con modelo, params, metrics
    for i, params in enumerate(param_grid):
        model = XGBRegressor(**xgb_base, **params)

        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            # eval_metric="rmse",
            verbose=False,
            # early_stopping_rounds=200,
        )

        y_val_pred = xgb_predict_best_iteration(model, X_val)
        m_val = eval_regression_extended(
            y_true=np.asarray(y_val, dtype=np.float64),
            y_pred=np.asarray(y_val_pred, dtype=np.float64),
            meta=meta_val,
            time_col="timestamp",
            group_col="symbol",
            quantile=0.1,
            periods_per_year=252,
        )
        score = pick_score(m_val)

        row = dict(
            horizon=horizon,
            lookback=lookback,
            grid_i=i,
            score=score,
            params=params,
            best_iteration=getattr(model, "best_iteration", None),
            val_metrics=m_val,
        )

        if (best is None) or (np.isfinite(score) and score > best["score"]):
            best = row

        print(f"  [{i+1:02d}/{len(param_grid)}] score={score: .6f} params={params} best_it={row['best_iteration']}")

    if best is None:
        raise RuntimeError("No se pudo seleccionar un modelo (best=None).")

    # Re-entrena NO; usamos el mejor ya entrenado. Para eso necesitamos volver a entrenar con sus params
    # (porque en el loop no guardamos el objeto modelo).
    best_params = best["params"]
    best_model = XGBRegressor(**xgb_base, **best_params)
    best_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        # eval_metric="rmse",
        verbose=False,
        # early_stopping_rounds=200,
    )

    # Eval final val/test
    y_val_pred = xgb_predict_best_iteration(best_model, X_val)
    y_test_pred = xgb_predict_best_iteration(best_model, X_test)

    metrics_val = eval_regression_extended(
        y_true=np.asarray(y_val, dtype=np.float64),
        y_pred=np.asarray(y_val_pred, dtype=np.float64),
        meta=meta_val,
        time_col="timestamp",
        group_col="symbol",
        quantile=0.1,
        periods_per_year=252,
    )
    metrics_test = eval_regression_extended(
        y_true=np.asarray(y_test, dtype=np.float64),
        y_pred=np.asarray(y_test_pred, dtype=np.float64),
        meta=meta_test,
        time_col="timestamp",
        group_col="symbol",
        quantile=0.1,
        periods_per_year=252,
    )

    # pred dfs
    pred_df_val = meta_val.copy()
    pred_df_val["y_true"] = np.asarray(y_val, dtype=np.float64)
    pred_df_val["y_pred"] = np.asarray(y_val_pred, dtype=np.float64)
    pred_df_val["error"] = pred_df_val["y_pred"] - pred_df_val["y_true"]

    pred_df_test = meta_test.copy()
    pred_df_test["y_true"] = np.asarray(y_test, dtype=np.float64)
    pred_df_test["y_pred"] = np.asarray(y_test_pred, dtype=np.float64)
    pred_df_test["error"] = pred_df_test["y_pred"] - pred_df_test["y_true"]

    # Pipeline sklearn (sin preprocesado extra)
    pipeline = Pipeline([("model", best_model)])

    # Flags (igual que en tu MLP)
    ind_flag = int(bool(include_indicators))
    econ_flag = int(bool(include_economic_indicators and econ_indicator_names is not None and len(econ_indicator_names) > 0))
    fmp_flag = int(bool(include_fmp))

    # Run dir (incluye algunos params clave)
    run_dir = (
        f"runs/xgb_{timeframe}_lb{lookback}_h{horizon}"
        f"_indicators{ind_flag}_econ{econ_flag}_fmp{fmp_flag}"
        f"_md{best_params['max_depth']}_lr{best_params['learning_rate']}"
        f"_mcw{best_params['min_child_weight']}_seed{xgb_base['random_state']}"
    )

    config = {
        "model": "xgboost_regressor",
        "timeframe": timeframe,
        "symbols": list(symbols),
        "lookback": int(lookback),
        "horizon": int(horizon),

        "base_feature_cols": list(base_feature_cols),
        "include_indicators": bool(include_indicators),
        "indicators_used": indicator_names if include_indicators else [],
        "include_economic_indicators": bool(include_economic_indicators),
        "econ_indicator_names": econ_indicator_names if include_economic_indicators else [],
        "include_fmp": bool(include_fmp),
        "fmp_prefix": "fmp",
        "fmp_feature_names": fmp_feature_names if include_fmp else [],
        "keep_fmp_asof_date": bool(keep_fmp_asof_date),

        "lags_by_feature": lags_by_feature,
        "default_lags": default_lags,

        "xgb_base": xgb_base,
        "xgb_params": best_params,
        "xgb_best_iteration": getattr(best_model, "best_iteration", None),

        "split": {
            "train_frac": 0.70,
            "val_frac": 0.15,
            "train_end_target_timestamp": str(pd.Timestamp(train_end)),
            "val_end_target_timestamp": str(pd.Timestamp(val_end)),
        },
    }

    metrics = {"val": metrics_val, "test": metrics_test}

    # Guardar artefacto
    saved_path = save_sklearn_pipeline_artifact(
        run_dir=run_dir,
        pipeline=pipeline,
        config=config,
        metrics=metrics,
        feature_names=feature_names,
        pred_df_val=pred_df_val,
        pred_df_test=pred_df_test,
        extra={
            "note": "XGBoost baseline seleccionado por métricas en validation",
        },
    )

    print("\nSaved:", saved_path)
    print("VAL key metrics (si existen):",
          "DailyRankIC_mean=", metrics_val.get("DailyRankIC_mean"),
          "QuantileSpread_sharpe=", metrics_val.get("QuantileSpread_sharpe"))

    print("TEST key metrics (si existen):",
          "DailyRankIC_mean=", metrics_test.get("DailyRankIC_mean"),
          "QuantileSpread_sharpe=", metrics_test.get("QuantileSpread_sharpe"))

    results.append({
        "run_dir": saved_path,
        "horizon": horizon,
        "lookback": lookback,
        "val_score": pick_score(metrics_val),
        "test_rankic": metrics_test.get("DailyRankIC_mean", np.nan),
        "test_spread_sharpe": metrics_test.get("QuantileSpread_sharpe", np.nan),
        "params": best_params,
        "best_iteration": getattr(best_model, "best_iteration", None),
    })
    saved_run_dirs.append(saved_path)

results_df = pd.DataFrame(results).sort_values(["horizon"])
results_df



HORIZON = 60 | lookback = 252
Shapes: train (110416, 513) val (25912, 513) test (32664, 513) | cut: 2022-09-19 2024-03-19
  [01/8] score= 0.052216 params={'colsample_bytree': 0.8, 'learning_rate': 0.03, 'max_depth': 4, 'min_child_weight': 1, 'reg_lambda': 1.0, 'subsample': 0.8} best_it=4
  [02/8] score= 0.052218 params={'colsample_bytree': 0.8, 'learning_rate': 0.03, 'max_depth': 4, 'min_child_weight': 5, 'reg_lambda': 1.0, 'subsample': 0.8} best_it=4
  [03/8] score=-0.038203 params={'colsample_bytree': 0.8, 'learning_rate': 0.03, 'max_depth': 6, 'min_child_weight': 1, 'reg_lambda': 1.0, 'subsample': 0.8} best_it=9
  [04/8] score= 0.066990 params={'colsample_bytree': 0.8, 'learning_rate': 0.03, 'max_depth': 6, 'min_child_weight': 5, 'reg_lambda': 1.0, 'subsample': 0.8} best_it=1
  [05/8] score= 0.080789 params={'colsample_bytree': 0.8, 'learning_rate': 0.07, 'max_depth': 4, 'min_child_weight': 1, 'reg_lambda': 1.0, 'subsample': 0.8} best_it=1
  [06/8] score= 0.080790 params={'colsampl

,run_dir,horizon,lookback,val_score,test_rankic,test_spread_sharpe,params,best_iteration
0,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,60,252,0.08079,0.061559,6.99665,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",1


# rerunning

In [11]:
# Toma uno de los run_dir guardados (por ejemplo el de horizon=60)
run_dir = 'runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0_md4_lr0.07_mcw5_seed42'
print("Checking:", run_dir)

pred_df, _meta = predict_artifact_to_compare(
    run_dir,
    conn=conn,
    enforce_timeframe_match=True,
    start='01-01-2024'
)

# Evalua de nuevo (debería coincidir con lo guardado si start/end no cambian)
m = eval_regression_extended(
    y_true=pred_df["y_true"].to_numpy(dtype=np.float64),
    y_pred=pred_df["y_pred"].to_numpy(dtype=np.float64),
    meta=pred_df[["symbol", "timestamp", "target_timestamp"]],
    time_col="timestamp",
    group_col="symbol",
    quantile=0.1,
    periods_per_year=252,
)
m


Checking: runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0_md4_lr0.07_mcw5_seed42


{'MAE': 0.11758746108858927,
 'MedianAE': 0.07705190777778625,
 'RMSE': 0.20587868658505612,
 'R2': 0.0019012529271140721,
 'HitRate(sign)': 0.6215021199273167,
 'HitRate(sign,deadzone)': 0.6215021199273167,
 'PearsonCorr(IC)': 0.06877866639215588,
 'SpearmanCorr(RankIC)': 0.14000297471034687,
 'AUC(Sign)': 0.5174542535518284,
 'N': 16510,
 'N_deadzone': 16510,
 'DailyIC_mean': 0.04941676579944929,
 'DailyIC_std': 0.17053508862943642,
 'DailyIC_tstat': 3.898520791417481,
 'DailyIC_frac_pos': 0.6243093922651933,
 'DailyIC_N': 181,
 'DailyRankIC_mean': 0.07584926982328118,
 'DailyRankIC_std': 0.17452842773564942,
 'DailyRankIC_tstat': 5.846884508654975,
 'DailyRankIC_frac_pos': 0.6685082872928176,
 'DailyRankIC_N': 181,
 'QuantileSpread_mean': 0.0592149688116876,
 'QuantileSpread_std': 0.07866485084356947,
 'QuantileSpread_sharpe': 11.949536268470148,
 'QuantileSpread_N': 202,
 'Conformal_qhat(alpha=0.1)': nan,
 'Conformal_coverage(alpha=0.1)': nan,
 'Conformal_avg_width(alpha=0.1)': nan

In [22]:
m = eval_regression_extended(pred_df['y_true'], pred_df['y_pred'], meta=_meta)
m_flip = eval_regression_extended(pred_df['y_true'], pred_df['y_pred'], meta=_meta)

print("RankIC:", m.get("DailyRankIC_mean"), "Spread:", m.get("QuantileSpread_mean"))
print("RankIC(flipped):", m_flip.get("DailyRankIC_mean"), "Spread(flipped):", m_flip.get("QuantileSpread_mean"))


RankIC: 0.021510059504387728 Spread: -0.0006339212595197327
RankIC(flipped): 0.021510059504387728 Spread(flipped): -0.0006339212595197327
